In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('ArunSol1').master('yarn').getOrCreate()

In [3]:
spark.sql('use indirameduri')

DataFrame[]

In [4]:
orderItemDF= spark.read.table('order_items')
orderItemDF.show()

+-------------+-------------------+---------------------+-------------------+-------------------+------------------------+
|order_item_id|order_item_order_id|order_item_product_id|order_item_quantity|order_item_subtotal|order_item_product_price|
+-------------+-------------------+---------------------+-------------------+-------------------+------------------------+
|            1|                  1|                  957|                  1|             299.98|                  299.98|
|            2|                  2|                 1073|                  1|             199.99|                  199.99|
|            3|                  2|                  502|                  5|              250.0|                    50.0|
|            4|                  2|                  403|                  1|             129.99|                  129.99|
|            5|                  4|                  897|                  2|              49.98|                   24.99|
|            6| 

In [42]:
ordersDF=spark.read.csv('/public/retail_db/orders').toDF('order_id' ,'order_date','order_cust_id','order_status')
ordersDF.show()

+--------+--------------------+-------------+---------------+
|order_id|          order_date|order_cust_id|   order_status|
+--------+--------------------+-------------+---------------+
|       1|2013-07-25 00:00:...|        11599|         CLOSED|
|       2|2013-07-25 00:00:...|          256|PENDING_PAYMENT|
|       3|2013-07-25 00:00:...|        12111|       COMPLETE|
|       4|2013-07-25 00:00:...|         8827|         CLOSED|
|       5|2013-07-25 00:00:...|        11318|       COMPLETE|
|       6|2013-07-25 00:00:...|         7130|       COMPLETE|
|       7|2013-07-25 00:00:...|         4530|       COMPLETE|
|       8|2013-07-25 00:00:...|         2911|     PROCESSING|
|       9|2013-07-25 00:00:...|         5657|PENDING_PAYMENT|
|      10|2013-07-25 00:00:...|         5648|PENDING_PAYMENT|
|      11|2013-07-25 00:00:...|          918| PAYMENT_REVIEW|
|      12|2013-07-25 00:00:...|         1837|         CLOSED|
|      13|2013-07-25 00:00:...|         9149|PENDING_PAYMENT|
|      1

In [65]:
from pyspark.sql.types import *
ordersDF.printSchema()
ordersDF.withColumn('order_date',ordersDF.order_date.cast(LongType()))

root
 |-- order_id: string (nullable = true)
 |-- order_date: string (nullable = true)
 |-- order_cust_id: string (nullable = true)
 |-- order_status: string (nullable = true)



DataFrame[order_id: string, order_date: bigint, order_cust_id: string, order_status: string]

In [66]:

joinedOrderDataDF = ordersDF.join(orderItemDF,ordersDF.order_id==orderItemDF.order_item_order_id)

joinedOrderDataDF.createOrReplaceTempView('order_joined')
joinedOrderDataDF.show()

+--------+--------------------+-------------+---------------+-------------+-------------------+---------------------+-------------------+-------------------+------------------------+
|order_id|          order_date|order_cust_id|   order_status|order_item_id|order_item_order_id|order_item_product_id|order_item_quantity|order_item_subtotal|order_item_product_price|
+--------+--------------------+-------------+---------------+-------------+-------------------+---------------------+-------------------+-------------------+------------------------+
|       1|2013-07-25 00:00:...|        11599|         CLOSED|            1|                  1|                  957|                  1|             299.98|                  299.98|
|       2|2013-07-25 00:00:...|          256|PENDING_PAYMENT|            2|                  2|                 1073|                  1|             199.99|                  199.99|
|       2|2013-07-25 00:00:...|          256|PENDING_PAYMENT|            3|          

In [67]:
spark.sql('desc order_joined').show(truncate=False)
joinedOrderDataDF.show()

+------------------------+---------+-------+
|col_name                |data_type|comment|
+------------------------+---------+-------+
|order_id                |string   |null   |
|order_date              |string   |null   |
|order_cust_id           |string   |null   |
|order_status            |string   |null   |
|order_item_id           |int      |null   |
|order_item_order_id     |int      |null   |
|order_item_product_id   |int      |null   |
|order_item_quantity     |tinyint  |null   |
|order_item_subtotal     |double   |null   |
|order_item_product_price|double   |null   |
+------------------------+---------+-------+

+--------+--------------------+-------------+---------------+-------------+-------------------+---------------------+-------------------+-------------------+------------------------+
|order_id|          order_date|order_cust_id|   order_status|order_item_id|order_item_order_id|order_item_product_id|order_item_quantity|order_item_subtotal|order_item_product_price|
+--

In [68]:
from pyspark.sql.functions import *
sqlResult = spark.sql("select to_date(from_unixtime(cast(order_date/1000 as bigint))) as order_formatted_date, order_status, cast(sum(order_item_subtotal) as DECIMAL (10,2)) as total_amount, count(distinct(order_id)) as total_orders from order_joined group by to_date(from_unixtime(cast(order_date/1000 as bigint))), order_status order by order_formatted_date desc,order_status,total_amount desc, total_orders");


In [69]:
sqlResult.show()

+--------------------+---------------+------------+------------+
|order_formatted_date|   order_status|total_amount|total_orders|
+--------------------+---------------+------------+------------+
|                null|       CANCELED|   696030.99|        1194|
|                null|         CLOSED|  3736048.79|        6301|
|                null|       COMPLETE| 11276933.69|       18965|
|                null|        ON_HOLD|  1864731.24|        3193|
|                null| PAYMENT_REVIEW|   357841.45|         608|
|                null|        PENDING|  3851881.28|        6385|
|                null|PENDING_PAYMENT|  7581671.05|       12581|
|                null|     PROCESSING|  4190636.76|        6900|
|                null|SUSPECTED_FRAUD|   766844.68|        1304|
+--------------------+---------------+------------+------------+

